In [ ]:
import time

In [ ]:
tips = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "127.0.0.1:9092").option("subscribe","tips").option("startingOffsets", "earliest").load()


In [ ]:
rawtipsQuery = tips.writeStream.trigger(processingTime='10 seconds').outputMode("append").format("console").start()


In [ ]:
!ls kafka

In [ ]:
!./kafka/pushKafkaData.sh

In [ ]:
time.sleep(3)

In [ ]:
rawVoterQuery.stop()


In [ ]:
kafkatipsDF = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "127.0.0.1:9092").option("subscribe", "tips").option("startingOffsets", "earliest").load()


In [ ]:
tipsStringDF = kafkatipsDF.selectExpr("CAST(value AS STRING)")


In [ ]:
stringTipsQuery = tipsStringDF.writeStream.trigger(processingTime='10 seconds').outputMode("append").format("console").option("truncate", "false").start()


In [ ]:
stringTipsQuery.stop()


In [ ]:
kafkaTipsDF = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "127.0.0.1:9092").option("subscribe", "tips").option("startingOffsets", "earliest").load()


from pyspark.sql.types import StringType, LongType, StructType, StructField

tipsSchema = StructType([StructField("gender",StringType(), True), StructField("total",LongType(), True), StructField("tip",LongType(), True)])



In [ ]:
from pyspark.sql.functions import from_json

tipsStatsDF = kafkatipsDF.select(from_json(kafkaVoterDF["value"].cast(StringType()), voterSchema).alias("voterJSON")).groupBy("voterJSON.gender", "voterJSON.party").count()

tipsStatsQuery = tipsStatsDF.writeStream.trigger(processingTime='1 minute').outputMode("complete").format("console").start()



In [ ]:
voterStatsQuery.stop()
